# Coordinate descent
CuML library can implement lasso and elastic net algorithms. The lasso model extends LinearRegression with L2 regularization and elastic net extends LinearRegression with a combination of L1 and L2 regularizations.
We see tremendous speed up for datasets with large number of rows and less number of rows. Furthermore, the MSE value for the cuML implementation is much smaller than the scikit-learn implementation for very small datasets.

In [1]:
# Select a particular GPU to run the notebook  
%env CUDA_VISIBLE_DEVICES=2
# import the required libraries
import numpy as np
import pandas as pd
import cudf
import os
from cuml import Lasso as cuLasso
from sklearn.linear_model import Lasso
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
from cuml.linear_model import ElasticNet as cuElasticNet
from sklearn.linear_model import ElasticNet

env: CUDA_VISIBLE_DEVICES=2


# Helper Functions

In [2]:
# calculate the time required by a cell to run
from timeit import default_timer

class Timer(object):
    def __init__(self):
        self._timer = default_timer
    
    def __enter__(self):
        self.start()
        return self

    def __exit__(self, *args):
        self.stop()

    def start(self):
        """Start the timer."""
        self.start = self._timer()

    def stop(self):
        """Stop the timer. Calculate the interval in seconds."""
        self.end = self._timer()
        self.interval = self.end - self.start

In [3]:
# check if the mortgage dataset is present and then extract the data from it, else just create a random dataset for regression 
import gzip
def load_data(nrows, ncols, cached = '/home/saljain/build-rapids/notebooks/cuml/data/mortgage.npy.gz'):
    #split the dataset in a 80:20 split
    train_rows = int(nrows*0.8)
    if os.path.exists(cached):
        print('use mortgage data')

        with gzip.open(cached) as f:
            X = np.load(f)
        # the 4th column is 'adj_remaining_months_to_maturity'
        # used as the label
        X = X[:,[i for i in range(X.shape[1]) if i!=4]]
        y = X[:,4:5]
        rindices = np.random.randint(0,X.shape[0]-1,nrows)
        X = X[rindices,:ncols]
        y = y[rindices]
        df_y_train = pd.DataFrame({'fea%d'%i:y[0:train_rows,i] for i in range(y.shape[1])})
        df_y_test = pd.DataFrame({'fea%d'%i:y[train_rows:,i] for i in range(y.shape[1])})
    else:
        print('use random data')
        X,y = make_regression(n_samples=nrows,n_features=ncols,n_informative=ncols, random_state=0)
        df_y_train = pd.DataFrame({'fea0':y[0:train_rows,]})
        df_y_test = pd.DataFrame({'fea0':y[train_rows:,]})

    df_X_train = pd.DataFrame({'fea%d'%i:X[0:train_rows,i] for i in range(X.shape[1])})
    df_X_test = pd.DataFrame({'fea%d'%i:X[train_rows:,i] for i in range(X.shape[1])})

    return df_X_train, df_X_test, df_y_train, df_y_test

# Obtain and convert the dataset

In [4]:
%%time
# nrows = number of samples
# ncols = number of features of each sample 
nrows = 90000
ncols = 500

#split the dataset into training and testing sets, in the ratio of 80:20 respectively
X_train, X_test, y_train, y_test = load_data(nrows,ncols)
print('training data',X_train.shape)
print('training label',y_train.shape)
print('testing data',X_test.shape)
print('testing label',y_test.shape)
print('label',y_test.shape)

use random data
training data (72000, 500)
training label (72000, 1)
testing data (18000, 500)
testing label (18000, 1)
label (18000, 1)
CPU times: user 9.48 s, sys: 880 ms, total: 10.4 s
Wall time: 3.23 s


In [5]:
%%time
# convert the pandas dataframe to cudf format
X_cudf = cudf.DataFrame.from_pandas(X_train)
X_cudf_test = cudf.DataFrame.from_pandas(X_test)
y_cudf = y_train.values
y_cudf = y_cudf[:,0]
y_cudf = cudf.Series(y_cudf)

CPU times: user 2.32 s, sys: 2.77 s, total: 5.1 s
Wall time: 5.12 s


### Define the model parameters

In [6]:
# learning rate = lr
# algorithm used in the model = algo
lr = 0.001
algo = 'cyclic'

### Lasso
The lasso model implemented in cuml allows the user to change the following parameter values:
1. alpha: regularizing constant that is multiplied with L1 to control the extent of regularization. (default = 1)
2. normalize: variable decides if the predictors in X will be normalized or not. (default = False)
3. fit_intercept: if set to True the model tries to center the data. (default = True)
4. max_iter: maximum number of iterations for training (fitting) the data to the model. (default = 1000)
5. tol: the tolerance for optimization. (default = 1e-3)
3. algorithm: the user can set the algorithm value as 'cyclic' or 'random'

The model accepts only numpy arrays or cudf dataframes as the input. In order to convert your dataset to cudf format please read the cudf documentation on https://rapidsai.github.io/projects/cudf/en/latest/. For additional information on the lasso model please refer to the documentation on https://rapidsai.github.io/projects/cuml/en/latest/index.html

#### Scikit-learn model for lasso

In [7]:
%%time
# use the sklearn lasso model to fit the training dataset 
skols = Lasso(alpha=np.array([lr]), fit_intercept = True, normalize = False, max_iter = 1000, selection=algo, tol=1e-10)
skols.fit(X_train, y_train)

CPU times: user 12.9 s, sys: 1.52 s, total: 14.5 s
Wall time: 412 ms


In [8]:
%%time
# calculate the mean squared error for the sklearn lasso model on the testing dataset
sk_predict = skols.predict(X_test)
error_sk = mean_squared_error(y_test,sk_predict)

CPU times: user 532 ms, sys: 52 ms, total: 584 ms
Wall time: 14.7 ms


#### CuML model for lasso

In [9]:
%%time
# run the cuml linear regression model to fit the training dataset 
cuols = cuLasso(alpha=np.array([lr]), fit_intercept = True, normalize = False, max_iter = 1000, selection=algo, tol=1e-10)
cuols.fit(X_cudf, y_cudf)

CPU times: user 8.11 s, sys: 652 ms, total: 8.76 s
Wall time: 1.37 s


In [10]:
%%time
# calculate the mean squared error of the testing dataset using the cuml linear regression model
cu_predict = cuols.predict(X_cudf_test).to_array()
error_cu = mean_squared_error(y_test,cu_predict)

CPU times: user 560 ms, sys: 8 ms, total: 568 ms
Wall time: 566 ms


In [11]:
# print the mean squared error of the sklearn and cuml model to compare the two
print("SKL MSE(y):")
print(error_sk)
print("CUML MSE(y):")
print(error_cu)

SKL MSE(y):
0.0005121767786469359
CUML MSE(y):
0.0005668184954986633


### Elastic Net
The elastic net model implemented in cuml contains the same parameters as the lasso model.
In addition to the variable values that can be altered in lasso, elastic net has another variable who's value can be changed
- l1_ratio: decides the ratio of amount of L1 and L2 regularization that would be applied to the model. When L1 ratio = 0, the model will have only L2 reqularization shall be applied to the model. (default = 0.5)

The model accepts only numpy arrays or cudf dataframes as the input. In order to convert your dataset to cudf format please read the cudf documentation on https://rapidsai.github.io/projects/cudf/en/latest/. For additional information on the lasso model please refer to the documentation on https://rapidsai.github.io/projects/cuml/en/latest/index.html

#### Scikit-learn model for elastic net

In [12]:
%%time
# use the sklearn linear regression model to fit the training dataset 
elastic_sk = ElasticNet(alpha=np.array([lr]), fit_intercept = True, normalize = False, max_iter = 1000, selection=algo, tol=1e-10)
elastic_sk.fit(X_train, y_train)

CPU times: user 12.8 s, sys: 1.35 s, total: 14.1 s
Wall time: 423 ms


In [13]:
%%time
# calculate the mean squared error of the sklearn linear regression model on the testing dataset
sk_predict_elas = elastic_sk.predict(X_test)
error_sk_elas = mean_squared_error(y_test,sk_predict_elas)

CPU times: user 492 ms, sys: 44 ms, total: 536 ms
Wall time: 13.4 ms


#### CuML model for elastic net

In [14]:
%%time
# run the cuml linear regression model to fit the training dataset 
elastic_cu = cuElasticNet(alpha=np.array([lr]), fit_intercept = True, normalize = False, max_iter = 1000, selection=algo, tol=1e-10)
elastic_cu.fit(X_cudf, y_cudf)

CPU times: user 7.82 s, sys: 512 ms, total: 8.33 s
Wall time: 864 ms


In [15]:
%%time
# calculate the mean squared error of the testing dataset using the cuml linear regression model
cu_predict_elas = elastic_cu.predict(X_cudf_test).to_array()
error_cu_elas = mean_squared_error(y_test,cu_predict_elas)

CPU times: user 576 ms, sys: 8 ms, total: 584 ms
Wall time: 581 ms


In [16]:
# print the mean squared error of the sklearn and cuml model to compare the two
print("SKL MSE(y):")
print(error_sk_elas)
print("CUML MSE(y):")
print(error_cu_elas)

SKL MSE(y):
0.4246530461077203
CUML MSE(y):
0.42472785735070406
